In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import time
import urllib3
import json

In [2]:
import os

url = 'https://www.bcn.cl/leychile/navegar/imprimir?idNorma=242302&idVersion=2022-04-29'

In [3]:
## Leyendo archivos

In [64]:
path = 'Textos/'
filelist = os.listdir(path)

In [65]:
filelist.sort()

In [66]:
filelist

['.ipynb_checkpoints',
 'Capitulo 01.txt',
 'Capitulo 02.txt',
 'Capitulo 03.txt',
 'Capitulo 04.txt',
 'Capitulo 05.txt',
 'Capitulo 06.txt',
 'Capitulo 07.txt',
 'Capitulo 08.txt',
 'Capitulo 09.txt',
 'Capitulo 10.txt',
 'Capitulo 11.txt',
 'Capitulo 12.txt',
 'Capitulo 13.txt',
 'Capitulo 14.txt',
 'Capitulo 15.txt',
 'Disposiciones transitorias.txt',
 'Promulgacion.txt']

Eliminando el capitulo de promulgación

In [67]:
filelist.remove('Promulgacion.txt')

In [68]:
filelist

['.ipynb_checkpoints',
 'Capitulo 01.txt',
 'Capitulo 02.txt',
 'Capitulo 03.txt',
 'Capitulo 04.txt',
 'Capitulo 05.txt',
 'Capitulo 06.txt',
 'Capitulo 07.txt',
 'Capitulo 08.txt',
 'Capitulo 09.txt',
 'Capitulo 10.txt',
 'Capitulo 11.txt',
 'Capitulo 12.txt',
 'Capitulo 13.txt',
 'Capitulo 14.txt',
 'Capitulo 15.txt',
 'Disposiciones transitorias.txt']

In [69]:
texto_total = ''

for file in filelist[1:]:
    with open(path+file, 'r', encoding = 'utf-8') as f:
        text = f.read()
        if text[-1:] != '\n':
            text = text+'\n'
        else:
            text = text
    
    texto_total = texto_total + text

serie = texto_total.split('\n')

In [70]:
df = pd.DataFrame({'texto':serie})

In [71]:
df['texto'] = df['texto'].str.strip()

In [72]:
df['capitulo'] = np.where(df['texto'].str.contains('Preámbulo'),df['texto'],np.nan)
df['capitulo'] = np.where(df['texto'].str.contains('DISPOSICIONES TRANSITORIAS'),df['texto'],df['capitulo'])
df['capitulo'] = np.where(df['texto'].str.contains('Capítulo'),df['texto'],df['capitulo'])
df['capitulo'] = np.where(df['texto'].str.contains('DECIMOSEXTA'),np.nan,df['capitulo'])
df['capitulo'] = np.where(df['texto'].str.contains('Los procesos iniciados, de oficio o a petición de parte, o que se iniciaren en la Corte Suprema'),np.nan,df['capitulo'])
df['capitulo'] = np.where(df['texto'].str.contains('Las contiendas de competencia actualmente trabadas ante la Corte Suprema'),np.nan,df['capitulo'])
df['capitulo'] = np.where(df['texto'].str.contains('En lo no previsto en este Capítulo, serán aplicables'),np.nan,df['capitulo'])


#Para asegurar que aparezca el capitulo IV
df['articulo'] = np.where(df['texto'].str.contains('Artículo'),df['texto'],np.nan)
df['texto'] = np.where(df['texto']=='', np.nan, df['texto'])
df['texto'] = np.where(df['texto']==df['articulo'], np.nan, df['texto'])
df['texto'] = np.where(df['texto']==df['capitulo'], np.nan, df['texto'])
# Agregar numeral y ordinal (??)


df['capitulo'].fillna(method='pad', inplace=True)
df['articulo'].fillna(method='pad', inplace=True)

In [73]:
df.dropna(subset = 'texto', inplace = True)

In [74]:
df['nro_capitulo'] = df['capitulo'].str.split(':', expand = True)[0]
df['nro_capitulo'] = df['nro_capitulo'].str.replace('# ','')
df['titulo_capitulo'] = df['capitulo'].str.split(':', expand = True)[1]
df['titulo_capitulo'] = df['titulo_capitulo'].str.strip()
df['articulo'] = df['articulo'].str.replace('### ','')

df['articulo'] = np.where(df['nro_capitulo']=='Convención Constitucional',np.nan,df['articulo'])
df['titulo_capitulo'] = np.where(df['titulo_capitulo'].isna(), df['nro_capitulo'], df['titulo_capitulo'])

In [75]:
df = df[['nro_capitulo','titulo_capitulo','articulo','texto']]
df.rename(columns = {'nro_capitulo':'capitulo'}, inplace = True)

In [76]:
df['capitulo'].value_counts()

DISPOSICIONES TRANSITORIAS    250
Capítulo V                    148
Capítulo III                  120
Capítulo XV                   108
Capítulo IV                    72
Capítulo XIV                   53
Capítulo VIII                  47
Capítulo II                    24
Capítulo VI                    23
Capítulo I                     16
Capítulo IX                    14
Capítulo VII                   10
Capítulo XI                     5
Capítulo X                      4
Capítulo XII                    4
Capítulo XIII                   4
Name: capitulo, dtype: int64

In [77]:
df

,capitulo,titulo_capitulo,articulo,texto
5,Capítulo I,BASES DE LA INSTITUCIONALIDAD,Artículo 1°.- Las personas nacen libres e igua...,La familia es el núcleo fundamental de la soci...
6,Capítulo I,BASES DE LA INSTITUCIONALIDAD,Artículo 1°.- Las personas nacen libres e igua...,El Estado reconoce y ampara a los grupos inter...
7,Capítulo I,BASES DE LA INSTITUCIONALIDAD,Artículo 1°.- Las personas nacen libres e igua...,El Estado está al servicio de la persona human...
8,Capítulo I,BASES DE LA INSTITUCIONALIDAD,Artículo 1°.- Las personas nacen libres e igua...,Es deber del Estado resguardar la seguridad na...
15,Capítulo I,BASES DE LA INSTITUCIONALIDAD,Artículo 3º.- El Estado de Chile es unitario.,La administración del Estado será funcional y ...
...,...,...,...,...
1706,DISPOSICIONES TRANSITORIAS,DISPOSICIONES TRANSITORIAS,VIGESIMA.- En tanto no se creen los tribunales...,"El procedimiento de solicitud, la exención de ..."
1707,DISPOSICIONES TRANSITORIAS,DISPOSICIONES TRANSITORIAS,VIGESIMA.- En tanto no se creen los tribunales...,Estarán impedidos de solicitar el retiro a que...
1708,DISPOSICIONES TRANSITORIAS,DISPOSICIONES TRANSITORIAS,VIGESIMA.- En tanto no se creen los tribunales...,Quienes hubieren hecho ejercicio del derecho e...
1709,DISPOSICIONES TRANSITORIAS,DISPOSICIONES TRANSITORIAS,VIGESIMA.- En tanto no se creen los tribunales...,"Sin perjuicio de lo anterior, quienes hubieren..."
